In [4]:
from pyprotolinc.models.model_annuity_runoff import AnnuityRunoffStates

In [5]:
for k in AnnuityRunoffStates:
    print(k)

AnnuityRunoffStates.DIS1
AnnuityRunoffStates.DEATH


In [7]:
k.name

'DEATH'

In [9]:
k.value


map_dict = AnnuityRunoffStates.to_std_outputs()
map_dict

{<ProbabilityVolumeResults.VOL_DIS1: 1>: <AnnuityRunoffStates.DIS1: 0>,
 <ProbabilityVolumeResults.VOL_DEATH: 3>: <AnnuityRunoffStates.DEATH: 1>,
 <ProbabilityVolumeResults.MV_DIS1_DEATH: 11>: (<AnnuityRunoffStates.DIS1: 0>,
  <AnnuityRunoffStates.DEATH: 1>)}

In [11]:
"afgh".startswith("ba")

False

In [20]:
for x in map_dict.items():
    x
    
x[0].name

'MV_DIS1_DEATH'

# Examples for Building a Model

In [1]:
from pyprotolinc.models.model_config import ModelBuilder
from pyprotolinc.assumptions.iohelpers import WorkbookTableReader
from pyprotolinc.models import MultiStateDisabilityStates, AnnuityRunoffStates

ImportError: cannot import name 'ModelBuilder' from 'pyprotolinc.models.model_config' (D:\programming\pyprotolinc\src\pyprotolinc\models\model_config.py)

### 1. Disability 2-State Model

In [2]:
path = r"D:\programming\py\PyMultiState\pyprotolinc\data\assumptions\base_assumption.xlsx"

with WorkbookTableReader(path) as file:
    print(file.sheet_names)

NameError: name 'WorkbookTableReader' is not defined

In [3]:
assumptions_spec = [
    (0, 3, 'MORTALITY (0->3)'),
    (0, 4, 'LAPSE (0->4)'),
    (0, 1, 'DIS1 (0->1)'),
    (1, 2, 'DIS_WORSEN (1->2)'),
    (1, 3, 'DIS_DEATH1 (1->3)'),
    (1, 0, 'REC1(1->0)'), 
    (0, 2, 'DIS2(0->2)'),
    (2, 1, 'DIS_IMPR (2->1)'),
    (2, 3, 'DIS_DEATH2 (2->3)'),
    (2, 0, 'REC2(2->0)')
]

In [4]:

model_builder = ModelBuilder(MultiStateDisabilityStates)

with WorkbookTableReader(path) as file:
    for assspec in assumptions_spec:
        table = file.read_sheet(assspec[2])
        model_builder.add_transition(assspec[0], assspec[1], table.rates_provider())
model = model_builder.build()

In [5]:
model.states_model, model.states_dim 

(<enum 'MultiStateDisabilityStates'>, 5)

In [6]:
model.rates_provider_matrix

[[None,
  <StandardRateProvider with factors (['age'])>,
  <StandardRateProvider with factors (['age'])>,
  <StandardRateProvider with factors (['age', 'gender'])>,
  <StandardRateProvider with factors (['calendaryear'])>],
 [<ConstantRateProvider with constant 0.2>,
  None,
  <StandardRateProvider with factors (['gender'])>,
  <StandardRateProvider with factors (['gender'])>,
  None],
 [<ConstantRateProvider with constant 0.01>,
  <ConstantRateProvider with constant 0.1>,
  None,
  <ConstantRateProvider with constant 0.2>,
  None],
 [None, None, None, None, None],
 [None, None, None, None, None]]

### 2. Annuity Model

In [7]:
model_builder = ModelBuilder(AnnuityRunoffStates)

In [8]:
from pyprotolinc.assumptions.dav2004r import DAV2004R

In [9]:
annuity_model = model_builder.add_transition(0, 1, DAV2004R().rates_provider())\
                             .build()

In [10]:
annuity_model.states_dim, annuity_model.states_model

(2, <enum 'AnnuityRunoffStates'>)

In [11]:
annuity_model.rates_provider_matrix

[[None,
  <StandardRateProvider with factors (['calendaryear', 'gender', 'age'])>],
 [None, None]]